# Using MCP Tools with aisuite

This notebook demonstrates how to use MCP (Model Context Protocol) servers with aisuite to give AI models access to external tools and data sources.

## What is MCP?

MCP (Model Context Protocol) is a standardized protocol that allows AI applications to connect to external data sources and tools. MCP servers expose tools, resources, and prompts that AI models can use.

## Prerequisites

Install aisuite with MCP support:
```bash
pip install 'aisuite[mcp]'
# Or install providers you need:
pip install 'aisuite[openai,mcp]'
```

You'll also need to install an MCP server. For this example, we'll use the filesystem server:
```bash
npm install -g @modelcontextprotocol/server-filesystem
```

In [ ]:
import os
from dotenv import load_dotenv
import aisuite as ai
from aisuite.mcp import MCPClient

# Load environment variables (API keys)
load_dotenv()

# Verify API key is set
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("Please set OPENAI_API_KEY environment variable")

## Example 1: Basic MCP Tool Usage

Let's connect to a filesystem MCP server and use it to read files.

In [ ]:
# Connect to the filesystem MCP server
# This gives the AI access to files in the specified directory
mcp_client = MCPClient(
    command="npx",
    args=["-y", "@modelcontextprotocol/server-filesystem", os.getcwd()]
)

print(f"Connected to MCP server: {mcp_client}")
print(f"\nAvailable tools:")
for tool in mcp_client.list_tools():
    print(f"  - {tool['name']}: {tool.get('description', 'No description')}")

In [ ]:
# Create aisuite client
client = ai.Client()

# Use MCP tools with aisuite
messages = [
    {"role": "user", "content": "Please read the README.md file and summarize what this project does."}
]

response = client.chat.completions.create(
    model="openai:gpt-4o",
    messages=messages,
    tools=mcp_client.get_callable_tools(),  # MCP tools work like regular Python functions!
    max_turns=3  # Allow multiple tool calls
)

print("\nAI Response:")
print(response.choices[0].message.content)

# View the tool interactions
print("\n" + "="*60)
print("Tool Call History:")
print("="*60)
for msg in response.choices[0].intermediate_messages:
    if msg.role == "assistant" and msg.tool_calls:
        for tool_call in msg.tool_calls:
            print(f"\nTool: {tool_call.function.name}")
            print(f"Arguments: {tool_call.function.arguments}")
    elif msg.role == "tool":
        print(f"Result: {msg.content[:200]}...")  # First 200 chars

## Example 2: Mixing MCP Tools with Python Functions

You can seamlessly mix MCP tools with regular Python functions!

In [ ]:
# Define a custom Python function
def get_current_time() -> str:
    """Get the current date and time."""
    from datetime import datetime
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

def calculate_word_count(text: str) -> int:
    """Calculate the number of words in a text.
    
    Args:
        text: The text to count words in
    """
    return len(text.split())

# Mix MCP tools and Python functions
all_tools = mcp_client.get_callable_tools() + [get_current_time, calculate_word_count]

messages = [
    {
        "role": "user", 
        "content": "What time is it? Also, read the README.md file and tell me how many words it contains."
    }
]

response = client.chat.completions.create(
    model="openai:gpt-4o",
    messages=messages,
    tools=all_tools,  # Both MCP and Python tools!
    max_turns=5
)

print(response.choices[0].message.content)

## Example 3: Using Specific MCP Tools

You can also select specific tools instead of using all of them.

In [ ]:
# Get only specific tools
read_file = mcp_client.get_tool("read_file")
list_directory = mcp_client.get_tool("list_directory")

# Use only these tools
messages = [
    {"role": "user", "content": "List all Python files in the current directory."}
]

response = client.chat.completions.create(
    model="openai:gpt-4o",
    messages=messages,
    tools=[read_file, list_directory],  # Only specific tools
    max_turns=2
)

print(response.choices[0].message.content)

## Example 4: Using MCP with Different Providers

MCP tools work with all aisuite providers!

## Example 4: Using MCP Config Dict Format (Simplified)

Instead of creating an MCPClient explicitly, you can pass MCP server configuration directly as a dict in the `tools` parameter! This is more convenient for simple use cases.

In [ ]:
# Try with different providers
providers = [
    "openai:gpt-4o",
    "anthropic:claude-3-5-sonnet-20240620",
]

messages = [
    {"role": "user", "content": "List the files in the current directory and tell me how many there are."}
]

for model in providers:
    print(f"\n{'='*60}")
    print(f"Provider: {model}")
    print(f"{'='*60}\n")
    
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=mcp_client.get_callable_tools(),
            max_turns=3
        )
        print(response.choices[0].message.content)
    except Exception as e:
        print(f"Error: {e}")
        print("Make sure you have the API key set and provider installed.")

## Example 5: Connecting to Multiple MCP Servers

You can connect to multiple MCP servers and combine their tools.

In [ ]:
# Connect to a second MCP server (example - adjust as needed)
# For demonstration, we'll create another filesystem server for a different directory

# Create a temp directory for demonstration
import tempfile
temp_dir = tempfile.mkdtemp()

# Second MCP client
mcp_client_2 = MCPClient(
    command="npx",
    args=["-y", "@modelcontextprotocol/server-filesystem", temp_dir]
)

# Combine tools from both servers
all_mcp_tools = (
    mcp_client.get_callable_tools() + 
    mcp_client_2.get_callable_tools()
)

print(f"Total tools available: {len(all_mcp_tools)}")

# Note: In practice, you might want to rename tools or use namespacing
# to avoid conflicts between servers with similar tools

## Cleanup

It's good practice to close MCP connections when done. You can also use MCPClient as a context manager.

In [ ]:
# Close the MCP connections
mcp_client.close()
mcp_client_2.close()

print("MCP connections closed.")

## Using Context Manager (Recommended)

The recommended way to use MCPClient is with a context manager:

In [ ]:
# Using context manager ensures proper cleanup
with MCPClient(
    command="npx",
    args=["-y", "@modelcontextprotocol/server-filesystem", os.getcwd()]
) as mcp:
    response = client.chat.completions.create(
        model="openai:gpt-4o",
        messages=[{"role": "user", "content": "How many files are in the current directory?"}],
        tools=mcp.get_callable_tools(),
        max_turns=2
    )
    print(response.choices[0].message.content)

# Connection is automatically closed after the with block

## Summary

Key takeaways:

1. **Easy Integration**: MCP tools work seamlessly with aisuite's existing tool system
2. **Mix and Match**: Combine MCP tools with regular Python functions
3. **Provider Agnostic**: Works with any aisuite provider (OpenAI, Anthropic, Google, etc.)
4. **Multiple Servers**: Connect to multiple MCP servers simultaneously
5. **Simple API**: Just `MCPClient()` → `get_callable_tools()` → pass to `tools=[]`

For more MCP servers, check out:
- https://github.com/modelcontextprotocol/servers
- Official MCP documentation: https://modelcontextprotocol.io/